# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
# to visualize progress
from tqdm import tqdm

#### Creating list of filepaths to process original event csv data files

In [2]:
print('working directory is', os.getcwd())
print("")

filepath = os.getcwd() + '/event_data'
file_path_list = glob.glob(os.path.join(filepath,'*.csv'))
print(file_path_list)

working directory is /home/workspace

['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_da

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [ ]:
query = "CREATE TABLE IF NOT EXISTS song_table "
query = query + "(artist text, song text, length float, sessionId int, \
itemInSession int, PRIMARY KEY (sessionId, itemInSession));"
session.execute(query)

In [9]:
# line_index: column_name
# 0 artist
# 1 firstName
# 2 gender
# 3 itemInSession
# 4 lastName
# 5 length
# 6 level
# 7 location
# 8 sessionId
# 9 song
# 10 userId

In [10]:
# Insert rows into song_table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO song_table (artist, song, length, sessionId, itemInSession) "
        query = query + "VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

6820it [00:11, 597.27it/s]


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
query = "SELECT artist, song, length FROM song_table WHERE sessionId = 338 AND itemInSession = 4;"
res = session.execute(query)
for r in res:
    print(r)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
query = "CREATE TABLE IF NOT EXISTS artist_table "
query = query + "(artist text, song text, firstName text, lastName text,itemInSession int, \
userId int, sessionId int, PRIMARY KEY ((userId, sessionId), itemInSession));"
session.execute(query)

In [13]:
# Insert rows into artist_table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO artist_table (artist, song, firstName, lastName, itemInSession, userId, sessionId) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (line[0], line[9], line[1], line[4], int(line[3]), int(line[10]), int(line[8])))

6820it [00:10, 662.18it/s]


In [14]:
query = "SELECT artist, song, firstName, lastName FROM artist_table \
WHERE userId = 10 AND sessionId = 182 ORDER BY itemInSession;"
res = session.execute(query)
for r in res:
    print(r)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [15]:
query = "CREATE TABLE IF NOT EXISTS user_table "
query = query + "(firstName text, lastName text, song text, userId int, PRIMARY KEY (song, userId));"
session.execute(query)

In [16]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO user_table (firstName, lastName, song, userId) "
        query = query + "VALUES (%s, %s, %s, %s);"
        session.execute( query, (line[1], line[4], line[9], int(line[10])) )

6820it [00:09, 685.59it/s]


In [17]:
query = "SELECT firstName, lastName FROM user_table WHERE song = 'All Hands Against His Own';"
row = session.execute(query)
for r in row:
    print(r)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables before closing out the sessions

In [18]:
query1 = "DROP TABLE IF EXISTS song_table;"
query2 = "DROP TABLE IF EXISTS artist_table;"
query3 = "DROP TABLE IF EXISTS user_table;"
session.execute(query1)
session.execute(query2)
session.execute(query3)

### Close the session and cluster connection¶

In [19]:
session.shutdown()
cluster.shutdown()